<a href="https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/runtime_config/ConfigurableLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Runtime Configurations


We want to configure our graph when calling it. For example, we want to specify LLM or system prompt to use at runtime _without polluting the graph state with these parameters_.

Adding runtime configuration, invloves a) specifying a schema for the configuration, b) adding the configuration to the nodes or conditional edges and c) passing the configuration to the graph

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_openai

## Speciyfing LLM at runtime

Below is a simple example which illustrates how to specify LLM at runtime using the Runtime Configuration functionality:

In [ ]:
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, StateGraph, START
from typing_extensions import TypedDict

from langchain.chat_models import init_chat_model
from langgrap.graph import MessageState

1) Specify the config schema

In [ ]:
class ConfigSchema(TypedDict):
    model: str

